# 4.7 Deriving New Variables

## This script contains the following points:

### 1. Create "price_label" column

### 2. Create "busiest_day" column

### 3. Update "busiest day" column to "busiest days" to identify order day of the week as "Busiest days", "Slowest days", or "Regularly busy"

### 4. Check values of new "busiest days" column for accuracy with obervations in markdown format

### 5. Create new colim "busiest_period_of_day" to identify time periods "Most orders", "Average orders", and "Fewest orders"

### 6. Print frequency of "busiest_period_of_day" column

### 7. Export dataframe as a pickle file to "Prepared Data" folder

# 01. Importing libraries

In [1]:
# Import libraries
import pandas as pd
import numpy as np
import os
import warnings

In [2]:
warnings.filterwarnings('ignore')

# 02. Importing data

In [3]:
df = pd.read_csv(r'C:\Users\Priya\27-02-2024 Instacart Basket Analysis\02 Data\Original Data\orders.csv', index_col = False)

In [4]:
path = r'C:\Users\Priya\27-02-2024 Instacart Basket Analysis'

In [5]:
path

'C:\\Users\\Priya\\27-02-2024 Instacart Basket Analysis'

In [6]:
# Import ords_prods_merge dataframe from Task 4.6.2
ords_prods_merge = pd.read_pickle(os.path.join(path, '02 Data', 'Prepared Data', 'ords_prods_merge.pkl'))

## 1. Create "price_label" column

In [7]:
# Create a subset of first million rows.
df = ords_prods_merge[:1000000]

In [8]:
df.shape

(1000000, 15)

In [9]:
# Define a function for price labels.
def price_label(row):

    if row['prices'] <= 5:
        return 'Low-range product'
    elif (row['prices'] > 5) and (row['prices'] <= 15):
        return 'Mid-range product'
    elif row['prices'] > 15:
        return 'High range'
    else: return 'Not enough data'

In [10]:
# Apply price range function. Note to self: Python can technically execute this, but it will suggest the loc() method instead.
df['price_range'] = df.apply(price_label, axis=1)

In [11]:
df['price_range'].value_counts(dropna = False)

price_range
Mid-range product    652638
Low-range product    338018
High range             9344
Name: count, dtype: int64

In [12]:
# Find most expensive product.
df['prices'].max()

24.5

In [13]:
# Use loc() method on df subset to apply price range function.
df.loc[df['prices'] > 15, 'price_range_loc'] = 'High-range product'

In [14]:
df.loc[(df['prices'] <= 15) & (df['prices'] > 5), 'price_range_loc'] = 'Mid-range product'

In [15]:
df.loc[df['prices'] <= 5, 'price_range_loc'] = 'Low-range product'

In [16]:
df['price_range_loc'].value_counts(dropna = False)

price_range_loc
Mid-range product     652638
Low-range product     338018
High-range product      9344
Name: count, dtype: int64

In [17]:
# Use loc() method on entire ords_prods_merge dataframe to apply price range function.
ords_prods_merge.loc[ords_prods_merge['prices'] > 15, 'price_range_loc'] = 'High-range product'

In [18]:
ords_prods_merge.loc[(ords_prods_merge['prices'] <= 15) & (ords_prods_merge['prices'] > 5), 'price_range_loc'] = 'Mid-range product'

In [19]:
ords_prods_merge.loc[ords_prods_merge['prices'] <= 5, 'price_range_loc'] = 'Low-range product'

In [20]:
ords_prods_merge['price_range_loc'].value_counts(dropna = False)

price_range_loc
Mid-range product     21861558
Low-range product     10130750
High-range product      412551
Name: count, dtype: int64

## 2. Create "busiest_day" column

In [21]:
# Change "order_dow" column name to "order_day_of_week".
ords_prods_merge.rename(columns = {'order_dow' : 'orders_day_of_week'}, inplace = True)

In [22]:
# Find frequency of orders_day_of_week.
ords_prods_merge['orders_day_of_week'].value_counts(dropna = False)

orders_day_of_week
0    6204182
1    5660230
6    4496490
2    4213830
5    4205791
3    3840534
4    3783802
Name: count, dtype: int64

In [23]:
# Use for-loop to determine whether orders are on "busiest day" (0 = Saturday), "least busy" (4 = Wednesday), or "regularly busy" (other days of the week).
result = []

for value in ords_prods_merge["orders_day_of_week"]:
    if value == 0:
        result.append("Busiest day")
    elif value == 4:
        result.append("Least busy")
    else:
        result.append("Regularly busy")

In [24]:
result

['Regularly busy',
 'Regularly busy',
 'Busiest day',
 'Regularly busy',
 'Least busy',
 'Regularly busy',
 'Regularly busy',
 'Regularly busy',
 'Regularly busy',
 'Regularly busy',
 'Regularly busy',
 'Regularly busy',
 'Least busy',
 'Regularly busy',
 'Regularly busy',
 'Regularly busy',
 'Regularly busy',
 'Regularly busy',
 'Regularly busy',
 'Regularly busy',
 'Regularly busy',
 'Regularly busy',
 'Regularly busy',
 'Regularly busy',
 'Regularly busy',
 'Busiest day',
 'Busiest day',
 'Busiest day',
 'Busiest day',
 'Busiest day',
 'Least busy',
 'Regularly busy',
 'Regularly busy',
 'Busiest day',
 'Regularly busy',
 'Regularly busy',
 'Least busy',
 'Regularly busy',
 'Busiest day',
 'Regularly busy',
 'Busiest day',
 'Least busy',
 'Busiest day',
 'Regularly busy',
 'Regularly busy',
 'Regularly busy',
 'Regularly busy',
 'Regularly busy',
 'Regularly busy',
 'Regularly busy',
 'Regularly busy',
 'Regularly busy',
 'Regularly busy',
 'Regularly busy',
 'Regularly busy',
 'Bus

In [25]:
# Create new column in ords_prods_merge to view results in context.
ords_prods_merge['busiest_day'] = result

In [26]:
ords_prods_merge['busiest_day'].value_counts(dropna = False)

busiest_day
Regularly busy    22416875
Busiest day        6204182
Least busy         3783802
Name: count, dtype: int64

## 3. Update "busiest day" column to "busiest days" to identify order day of the week as "Busiest days", "Slowest days", or "Regularly busy"

In [27]:
# Create column "busiest_days" from "orders_day_of_week" to identify two busiest days (0, 1), slowest days (4,3), or regularly busy (all other days).
result_2 = []

for value in ords_prods_merge["orders_day_of_week"]:
    if value == 0 or value == 1:
        result_2.append("Busiest days")
    elif value == 4 or value == 3:
        result_2.append("Slowest days")
    else:
        result_2.append("Regularly busy")

In [28]:
result_2

['Regularly busy',
 'Regularly busy',
 'Busiest days',
 'Slowest days',
 'Slowest days',
 'Busiest days',
 'Regularly busy',
 'Slowest days',
 'Busiest days',
 'Busiest days',
 'Regularly busy',
 'Slowest days',
 'Slowest days',
 'Regularly busy',
 'Slowest days',
 'Regularly busy',
 'Regularly busy',
 'Regularly busy',
 'Busiest days',
 'Busiest days',
 'Regularly busy',
 'Busiest days',
 'Busiest days',
 'Busiest days',
 'Busiest days',
 'Busiest days',
 'Busiest days',
 'Busiest days',
 'Busiest days',
 'Busiest days',
 'Slowest days',
 'Regularly busy',
 'Busiest days',
 'Busiest days',
 'Regularly busy',
 'Regularly busy',
 'Slowest days',
 'Busiest days',
 'Busiest days',
 'Busiest days',
 'Busiest days',
 'Slowest days',
 'Busiest days',
 'Busiest days',
 'Busiest days',
 'Slowest days',
 'Regularly busy',
 'Regularly busy',
 'Busiest days',
 'Busiest days',
 'Busiest days',
 'Slowest days',
 'Regularly busy',
 'Busiest days',
 'Regularly busy',
 'Busiest days',
 'Busiest days',

## 4. Check values of new "busiest days" column for accuracy with obervations in markdown format

In [29]:
# Create new 'busiest_days' column in ords_prods_merge to view results in context.
ords_prods_merge['busiest_days'] = result_2

In [30]:
ords_prods_merge['busiest_days'].value_counts(dropna = False)

busiest_days
Regularly busy    12916111
Busiest days      11864412
Slowest days       7624336
Name: count, dtype: int64

In [31]:
# Check output of ords_prods_merge with new "busiest days" column.
ords_prods_merge.head()

,product_id,product_name,aisle_id,department_id,prices,order_id,user_id,order_number,orders_day_of_week,order_hour_of_day,days_since_prior_order,first_order,add_to_cart_order,reordered,_merge,price_range_loc,busiest_day,busiest_days
0,1,Chocolate Sandwich Cookies,61,19,5.8,3139998,138,28,6,11,3.0,False,5,0,both,Mid-range product,Regularly busy,Regularly busy
1,1,Chocolate Sandwich Cookies,61,19,5.8,1977647,138,30,6,17,20.0,False,1,1,both,Mid-range product,Regularly busy,Regularly busy
2,1,Chocolate Sandwich Cookies,61,19,5.8,389851,709,2,0,21,6.0,False,20,0,both,Mid-range product,Busiest day,Busiest days
3,1,Chocolate Sandwich Cookies,61,19,5.8,652770,764,1,3,13,NaN,True,10,0,both,Mid-range product,Regularly busy,Slowest days
4,1,Chocolate Sandwich Cookies,61,19,5.8,1813452,764,3,4,17,9.0,False,11,1,both,Mid-range product,Least busy,Slowest days


### The total value counts for the ords_prods_merge dataframe is equal to the total sum of all groupings (i.e. "Regularly busy" + "Busiest days" + "Slowest days"). In addition, the listed sums for each label match the sum totals for the qualifying days of the week (e.g. Total "Busiest days" = 11864412, which matches orders_day_of_week 0 + 1, or 6204182 + 5660230).

## 5. Create new colim "busiest_period_of_day" to identify time periods "Most orders", "Average orders", and "Fewest orders"

In [32]:
# Check value counts in "order_hour_of_day" column.
ords_prods_merge['order_hour_of_day'].value_counts()

order_hour_of_day
10    2761760
11    2736140
14    2689136
15    2662144
13    2660954
12    2618532
16    2535202
9     2454203
17    2087654
8     1718118
18    1636502
19    1258305
20     976156
7      891054
21     795637
22     634225
23     402316
6      290493
0      218769
1      115700
5       87961
2       69375
4       53242
3       51281
Name: count, dtype: int64

### The value counts listed above will be split into equal thirds for the following labels: "Most orders" = 10, 11, 14, 15, 13, 12, 16, and 9; "Fewest orders" = 23, 6, 0, 1, 5, 2, 4, and 3; "Average orders" = all remaining values in order_hour_of_day.

In [33]:
# Create for-loop if statement labeling periods of time as "Most orders", "Average orders", and "Fewest orders".
result_3 = []

for value in ords_prods_merge["order_hour_of_day"]:
    if value in [10, 11, 14, 15, 13, 12, 16, 9]:
        result_3.append("Most orders")
    elif value in [23, 6, 0, 1, 5, 2, 4, 3]:
        result_3.append("Fewest orders")
    else:
        result_3.append("Average orders")

In [34]:
result_3

['Most orders',
 'Average orders',
 'Average orders',
 'Most orders',
 'Average orders',
 'Average orders',
 'Most orders',
 'Most orders',
 'Average orders',
 'Most orders',
 'Most orders',
 'Most orders',
 'Most orders',
 'Most orders',
 'Most orders',
 'Average orders',
 'Average orders',
 'Fewest orders',
 'Average orders',
 'Fewest orders',
 'Fewest orders',
 'Fewest orders',
 'Fewest orders',
 'Most orders',
 'Most orders',
 'Average orders',
 'Average orders',
 'Average orders',
 'Average orders',
 'Average orders',
 'Average orders',
 'Average orders',
 'Fewest orders',
 'Average orders',
 'Most orders',
 'Most orders',
 'Average orders',
 'Most orders',
 'Most orders',
 'Most orders',
 'Average orders',
 'Most orders',
 'Most orders',
 'Most orders',
 'Average orders',
 'Most orders',
 'Average orders',
 'Most orders',
 'Most orders',
 'Most orders',
 'Most orders',
 'Most orders',
 'Most orders',
 'Most orders',
 'Average orders',
 'Average orders',
 'Most orders',
 'Most ord

In [35]:
# Create new column "busiest_period_of_day" in ords_prods_merge.
ords_prods_merge['busiest_period_of_day'] = result_3

## 6. Print frequency of "busiest_period_of_day" column

In [36]:
# Print value counts in "busiest_period_of_day" column.
ords_prods_merge['busiest_period_of_day'].value_counts(dropna = False)

busiest_period_of_day
Most orders       21118071
Average orders     9997651
Fewest orders      1289137
Name: count, dtype: int64

## 7. Export dataframe as a pickle file to "Prepared Data" folder

In [37]:
ords_prods_merge.to_pickle(os.path.join(path, '02 Data', 'Prepared Data', 'ords_prods_merge.pkl'))